# Лабораторная работа №3: Классификация с использованием kNN

## Цель работы
Реализовать алгоритм k-ближайших соседей (kNN) для бинарной и многоклассовой классификации на датасете `fake_bills.csv`. Выполнить предобработку данных, нормализацию, отбор признаков, оценку метрик и кросс-валидацию.

## Описание датасета
- **Название**: fake_bills.csv
- **Признаки**: diagonal, height_left, height_right, margin_low, margin_up, length (числовые)
- **Целевая переменная (бинарная)**: is_genuine (True/False)
- **Размер**: 1500 строк
- Для многоклассовой классификации создаются 3 класса на основе квантилей признака `length`: Short, Medium, Long.

## Этапы работы
1. Загрузка и описание датасета.
2. Бинарная классификация:
   - Предобработка (обработка пропусков).
   - kNN без нормализации (k=5).
   - Нормализация данных.
   - kNN с нормализацией.
   - Отбор признаков по корреляции.
   - kNN с отобранными признаками.
   - Кросс-валидация для выбора k.
3. Многоклассовая классификация:
   - Создание классов.
   - Предобработка и нормализация.
   - kNN с нормализацией.
   - Отбор признаков.
   - kNN с отобранными признаками.
   - Кросс-валидация и выбор лучшего k.
   - Построение матрицы ошибок для лучшего классификатора.

In [1]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Установка стиля для графиков
%matplotlib inline

## 1. Загрузка и описание датасета

Загружаем датасет `fake_bills.csv` и выводим основную информацию о нем: описание признаков, размер, первые строки и наличие пропусков.

In [2]:
# Загрузка датасета
df = pd.read_csv('fake_bills.csv', sep=';')

# Описание датасета
print("Описание датасета:")
print("Датасет содержит измерения банкнот для определения их подлинности.")
print("Признаки: diagonal, height_left, height_right, margin_low, margin_up, length")
print("Целевая переменная (бинарная): is_genuine (True/False)")
print(f"Размер: {df.shape}")
print("\nПервые строки:")
print(df.head())

# Проверка пропусков
print("\nПропущенные значения:\n", df.isnull().sum())

Описание датасета:
Датасет содержит измерения банкнот для определения их подлинности.
Признаки: diagonal, height_left, height_right, margin_low, margin_up, length
Целевая переменная (бинарная): is_genuine (True/False)
Размер: (1500, 7)

Первые строки:
   is_genuine  diagonal  height_left  height_right  margin_low  margin_up  \
0        True    171.81       104.86        104.95        4.52       2.89   
1        True    171.46       103.36        103.66        3.77       2.99   
2        True    172.69       104.48        103.50        4.40       2.94   
3        True    171.36       103.91        103.94        3.62       3.01   
4        True    171.73       104.28        103.46        4.04       3.48   

   length  
0  112.83  
1  113.09  
2  113.16  
3  113.51  
4  112.54  

Пропущенные значения:
 is_genuine       0
diagonal         0
height_left      0
height_right     0
margin_low      37
margin_up        0
length           0
dtype: int64


## 2. Бинарная классификация

### 2.1 Предобработка данных

Обрабатываем пропущенные значения (заполняем медианой для `margin_low`). Проверяем, что пропусков не осталось. Разделяем данные на признаки (X) и целевую переменную (y).

In [3]:
# Предобработка данных
print("\nПредобработка для бинарной классификации:")
# Заполнение пропусков в 'margin_low' медианой
df['margin_low'] = df['margin_low'].fillna(df['margin_low'].median())
# Проверка пропусков после заполнения
print("Пропущенные значения после заполнения:\n", df.isnull().sum())

# Разделение на признаки и целевую переменную
X = df.drop('is_genuine', axis=1)
y = df['is_genuine']
# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Предобработка для бинарной классификации:
Пропущенные значения после заполнения:
 is_genuine      0
diagonal        0
height_left     0
height_right    0
margin_low      0
margin_up       0
length          0
dtype: int64


### 2.2 kNN без нормализации

Обучаем kNN c ( k=5 ) на ненормализованных данных и вычисляем метрики: точность, точность (precision), полноту (recall), F1-меру.

In [4]:
# Обучение kNN без нормализации
k = 5
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

# Вычисление метрик
print("\nМетрики без нормализации:")
print(f"Точность (Accuracy): {accuracy_score(y_test, y_pred):.4f}")
print(f"Точность (Precision): {precision_score(y_test, y_pred):.4f}")
print(f"Полнота (Recall): {recall_score(y_test, y_pred):.4f}")
print(f"F1-мера: {f1_score(y_test, y_pred):.4f}")
print("\nОтчет классификации:")
print(classification_report(y_test, y_pred))


Метрики без нормализации:
Точность (Accuracy): 0.9867
Точность (Precision): 0.9794
Полнота (Recall): 1.0000
F1-мера: 0.9896

Отчет классификации:
              precision    recall  f1-score   support

       False       1.00      0.96      0.98       110
        True       0.98      1.00      0.99       190

    accuracy                           0.99       300
   macro avg       0.99      0.98      0.99       300
weighted avg       0.99      0.99      0.99       300



### 2.3 Нормализация данных

Нормализуем признаки с помощью `StandardScaler` и повторяем обучение kNN.

In [5]:
# Нормализация данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Обучение kNN с нормализацией
knn_scaled = KNeighborsClassifier(n_neighbors=k)
knn_scaled.fit(X_train_scaled, y_train)
y_pred_scaled = knn_scaled.predict(X_test_scaled)

# Метрики с нормализацией
print("\nМетрики с нормализацией:")
print(f"Точность (Accuracy): {accuracy_score(y_test, y_pred_scaled):.4f}")
print(f"Точность (Precision): {precision_score(y_test, y_pred_scaled):.4f}")
print(f"Полнота (Recall): {recall_score(y_test, y_pred_scaled):.4f}")
print(f"F1-мера: {f1_score(y_test, y_pred_scaled):.4f}")
print("\nОтчет классификации:")
print(classification_report(y_test, y_pred_scaled))

# Сравнение метрик
print("\nСравнение метрик:")
print("Нормализация улучшает производительность kNN за счет приведения признаков к единому масштабу.")


Метрики с нормализацией:
Точность (Accuracy): 0.9833
Точность (Precision): 0.9744
Полнота (Recall): 1.0000
F1-мера: 0.9870

Отчет классификации:
              precision    recall  f1-score   support

       False       1.00      0.95      0.98       110
        True       0.97      1.00      0.99       190

    accuracy                           0.98       300
   macro avg       0.99      0.98      0.98       300
weighted avg       0.98      0.98      0.98       300


Сравнение метрик:
Нормализация улучшает производительность kNN за счет приведения признаков к единому масштабу.


### 2.4 Отбор признаков

Используем метод отбора признаков на основе корреляции:
- Зануляем диагональ матрицы корреляции.
- Применяем порог ( T_{corr} = 0.3 ) для фильтрации корреляций.
- Суммируем корреляции для каждого признака.
- Сортируем по убыванию и применяем порог ( T_{filter} = 0 ).
- Строим график важности признаков.

In [6]:
# Матрица корреляции
corr_matrix = X.corr().abs()
# Зануляем диагональ
np.fill_diagonal(corr_matrix.values, 0)
# Порог корреляции
T_corr = 0.3
# Фильтрация корреляций
high_corr = corr_matrix.where(corr_matrix > T_corr, 0)
# Сумма корреляций для каждого признака
feature_importance = high_corr.sum()
print("\nВажность признаков (сумма корреляций > T_corr):")
print(feature_importance)

# Сортировка
F_sorted = feature_importance.sort_values(ascending=False)
# Построение графика
plt.figure(figsize=(10, 6))
F_sorted.plot(kind='bar')
plt.title('Важность признаков (бинарная классификация)')
plt.ylabel('Сумма корреляций > T_corr')
plt.savefig('feature_importance_binary.png')
plt.close()

# Порог фильтрации
T_filter = 0
selected_features = F_sorted[F_sorted > T_filter].index.tolist()
print(f"Отобранные признаки: {selected_features}")

# Фильтрация датасета
X_filtered = X[selected_features]
X_filtered_scaled = scaler.fit_transform(X_filtered)
X_train_filt, X_test_filt, y_train, y_test = train_test_split(X_filtered_scaled, y, test_size=0.2, random_state=42)


Важность признаков (сумма корреляций > T_corr):
diagonal        0.000000
height_left     0.621205
height_right    1.093876
margin_low      1.771914
margin_up       1.254636
length          1.902585
dtype: float64
Отобранные признаки: ['length', 'margin_low', 'margin_up', 'height_right', 'height_left']


### 2.5 kNN с отобранными признаками

Обучаем kNN на отобранных признаках и сравниваем метрики.

In [7]:
# Обучение kNN с отобранными признаками
knn_filt = KNeighborsClassifier(n_neighbors=k)
knn_filt.fit(X_train_filt, y_train)
y_pred_filt = knn_filt.predict(X_test_filt)

# Метрики
print("\nМетрики с отобранными признаками:")
print(f"Точность (Accuracy): {accuracy_score(y_test, y_pred_filt):.4f}")
print(f"Точность (Precision): {precision_score(y_test, y_pred_filt):.4f}")
print(f"Полнота (Recall): {recall_score(y_test, y_pred_filt):.4f}")
print(f"F1-мера: {f1_score(y_test, y_pred_filt):.4f}")
print("\nОтчет классификации:")
print(classification_report(y_test, y_pred_filt))

# Сравнение
print("\nСравнение результатов до и после фильтрации:")
print("Фильтрация может улучшить производительность за счет удаления неинформативных признаков.")


Метрики с отобранными признаками:
Точность (Accuracy): 0.9867
Точность (Precision): 0.9794
Полнота (Recall): 1.0000
F1-мера: 0.9896

Отчет классификации:
              precision    recall  f1-score   support

       False       1.00      0.96      0.98       110
        True       0.98      1.00      0.99       190

    accuracy                           0.99       300
   macro avg       0.99      0.98      0.99       300
weighted avg       0.99      0.99      0.99       300


Сравнение результатов до и после фильтрации:
Фильтрация может улучшить производительность за счет удаления неинформативных признаков.


### 2.6 Кросс-валидация

Проводим кросс-валидацию для ( k ) от 1 до 20 и строим график точности на обучающей и тестовой выборках.

In [8]:
# Кросс-валидация для выбора k
k_values = range(1, 21)
train_scores = []
test_scores = []

for k in k_values:
    knn_cv = KNeighborsClassifier(n_neighbors=k)
    # Кросс-валидация на 5 фолдах
    cv_scores = cross_val_score(knn_cv, X_filtered_scaled, y, cv=5, scoring='accuracy')
    test_scores.append(cv_scores.mean())
    # Точность на обучающей выборке
    knn_cv.fit(X_train_filt, y_train)
    train_scores.append(knn_cv.score(X_train_filt, y_train))

# Построение графика
plt.figure(figsize=(10, 6))
plt.plot(k_values, train_scores, label='Точность на обучении')
plt.plot(k_values, test_scores, label='Точность на тесте')
plt.xlabel('k')
plt.ylabel('Точность')
plt.title('Точность на обучении и тесте в зависимости от k (бинарная классификация)')
plt.legend()
plt.grid(True)
plt.savefig('knn_accuracy_vs_k_binary.png')
plt.close()

## 3. Многоклассовая классификация

### 3.1 Создание классов

Создаем 3 класса (Short, Medium, Long) на основе квантилей признака `length`. Исключаем `length` из признаков, чтобы избежать утечки данных.

In [9]:
# Создание классов на основе квантилей length
df['length_class'] = pd.qcut(df['length'], q=3, labels=['Short', 'Medium', 'Long'])
print("\nОписание многоклассовой классификации:")
print("Созданы 3 класса на основе квантилей length: Short, Medium, Long")
print("Распределение классов:\n", df['length_class'].value_counts())

# Разделение данных
X_multi = df.drop(['is_genuine', 'length_class', 'length'], axis=1)
y_multi = df['length_class']
X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(X_multi, y_multi, test_size=0.2, random_state=42)


Описание многоклассовой классификации:
Созданы 3 класса на основе квантилей length: Short, Medium, Long
Распределение классов:
 length_class
Medium    508
Short     500
Long      492
Name: count, dtype: int64


### 3.2 Нормализация и kNN

Нормализуем данные и обучаем kNN с ( k=5 ).

In [10]:
# Нормализация
X_multi_scaled = scaler.fit_transform(X_multi)
X_train_multi_scaled, X_test_multi_scaled, y_train_multi, y_test_multi = train_test_split(X_multi_scaled, y_multi, test_size=0.2, random_state=42)

# Обучение kNN
knn_multi = KNeighborsClassifier(n_neighbors=k)
knn_multi.fit(X_train_multi_scaled, y_train_multi)
y_pred_multi = knn_multi.predict(X_test_multi_scaled)

# Метрики
print("\nМетрики многоклассовой классификации:")
print(f"Точность (Accuracy): {accuracy_score(y_test_multi, y_pred_multi):.4f}")
print(f"Точность (Precision): {precision_score(y_test_multi, y_pred_multi, average='weighted'):.4f}")
print(f"Полнота (Recall): {recall_score(y_test_multi, y_pred_multi, average='weighted'):.4f}")
print(f"F1-мера: {f1_score(y_test_multi, y_pred_multi, average='weighted'):.4f}")
print("\nОтчет классификации:")
print(classification_report(y_test_multi, y_pred_multi))


Метрики многоклассовой классификации:
Точность (Accuracy): 0.5900
Точность (Precision): 0.5869
Полнота (Recall): 0.5900
F1-мера: 0.5816

Отчет классификации:
              precision    recall  f1-score   support

        Long       0.41      0.52      0.46        86
      Medium       0.49      0.35      0.40       110
       Short       0.84      0.90      0.87       104

    accuracy                           0.59       300
   macro avg       0.58      0.59      0.58       300
weighted avg       0.59      0.59      0.58       300



### 3.3 Отбор признаков

Применяем тот же метод отбора признаков для многоклассовой задачи.

In [11]:
# Матрица корреляции
corr_matrix_multi = X_multi.corr().abs()
np.fill_diagonal(corr_matrix_multi.values, 0)
high_corr_multi = corr_matrix_multi.where(corr_matrix_multi > T_corr, 0)
feature_importance_multi = high_corr_multi.sum()
print("\nВажность признаков (сумма корреляций > T_corr, многоклассовая):")
print(feature_importance_multi)

# Сортировка и график
F_sorted_multi = feature_importance_multi.sort_values(ascending=False)
plt.figure(figsize=(10, 6))
F_sorted_multi.plot(kind='bar')
plt.title('Важность признаков (многоклассовая классификация)')
plt.ylabel('Сумма корреляций > T_corr')
plt.savefig('feature_importance_multi.png')
plt.close()

# Отбор признаков
selected_features_multi = F_sorted_multi[F_sorted_multi > T_filter].index.tolist()
print(f"Отобранные признаки: {selected_features_multi}")

# Фильтрация
X_multi_filtered = X_multi[selected_features_multi]
X_multi_filt_scaled = scaler.fit_transform(X_multi_filtered)
X_train_multi_filt, X_test_multi_filt, y_train_multi, y_test_multi = train_test_split(X_multi_filt_scaled, y_multi, test_size=0.2, random_state=42)


Важность признаков (сумма корреляций > T_corr, многоклассовая):
diagonal        0.000000
height_left     0.300342
height_right    0.692125
margin_low      1.112519
margin_up       0.734061
dtype: float64
Отобранные признаки: ['margin_low', 'margin_up', 'height_right', 'height_left']


### 3.4 kNN с отобранными признаками

Обучаем kNN на отобранных признаках.

In [12]:
# Обучение kNN
knn_multi_filt = KNeighborsClassifier(n_neighbors=k)
knn_multi_filt.fit(X_train_multi_filt, y_train_multi)
y_pred_multi_filt = knn_multi_filt.predict(X_test_multi_filt)

# Метрики
print("\nМетрики с отобранными признаками (многоклассовая):")
print(f"Точность (Accuracy): {accuracy_score(y_test_multi, y_pred_multi_filt):.4f}")
print(f"Точность (Precision): {precision_score(y_test_multi, y_pred_multi_filt, average='weighted'):.4f}")
print(f"Полнота (Recall): {recall_score(y_test_multi, y_pred_multi_filt, average='weighted'):.4f}")
print(f"F1-мера: {f1_score(y_test_multi, y_pred_multi_filt, average='weighted'):.4f}")
print("\nОтчет классификации:")
print(classification_report(y_test_multi, y_pred_multi_filt))

# Сравнение
print("\nСравнение результатов:")
print("Фильтрация может улучшить производительность за счет удаления лишних признаков.")


Метрики с отобранными признаками (многоклассовая):
Точность (Accuracy): 0.5667
Точность (Precision): 0.5625
Полнота (Recall): 0.5667
F1-мера: 0.5594

Отчет классификации:
              precision    recall  f1-score   support

        Long       0.37      0.47      0.41        86
      Medium       0.46      0.34      0.39       110
       Short       0.83      0.89      0.86       104

    accuracy                           0.57       300
   macro avg       0.55      0.57      0.55       300
weighted avg       0.56      0.57      0.56       300


Сравнение результатов:
Фильтрация может улучшить производительность за счет удаления лишних признаков.


### 3.5 Кросс-валидация и лучший классификатор

Проводим кросс-валидацию для выбора оптимального ( k ), обучаем лучший классификатор и строим матрицу ошибок.

In [13]:
# Кросс-валидация
train_scores_multi = []
test_scores_multi = []

for k in k_values:
    knn_cv_multi = KNeighborsClassifier(n_neighbors=k)
    cv_scores = cross_val_score(knn_cv_multi, X_multi_filt_scaled, y_multi, cv=5, scoring='accuracy')
    test_scores_multi.append(cv_scores.mean())
    knn_cv_multi.fit(X_train_multi_filt, y_train_multi)
    train_scores_multi.append(knn_cv_multi.score(X_train_multi_filt, y_train_multi))

# График
plt.figure(figsize=(10, 6))
plt.plot(k_values, train_scores_multi, label='Точность на обучении')
plt.plot(k_values, test_scores_multi, label='Точность на тесте')
plt.xlabel('k')
plt.ylabel('Точность')
plt.title('Точность на обучении и тесте в зависимости от k (многоклассовая классификация)')
plt.legend()
plt.grid(True)
plt.savefig('knn_accuracy_vs_k_multi.png')
plt.close()

# Лучший k
best_k = k_values[np.argmax(test_scores_multi)]
print(f"\nЛучшее k для многоклассовой классификации: {best_k}")

# Обучение лучшего классификатора
knn_best = KNeighborsClassifier(n_neighbors=best_k)
knn_best.fit(X_train_multi_filt, y_train_multi)
y_pred_best = knn_best.predict(X_test_multi_filt)

# Метрики
print("\nМетрики лучшего классификатора:")
print(f"Точность (Accuracy): {accuracy_score(y_test_multi, y_pred_best):.4f}")
print(f"Точность (Precision): {precision_score(y_test_multi, y_pred_best, average='weighted'):.4f}")
print(f"Полнота (Recall): {recall_score(y_test_multi, y_pred_best, average='weighted'):.4f}")
print(f"F1-мера: {f1_score(y_test_multi, y_pred_best, average='weighted'):.4f}")
print("\nОтчет классификации:")
print(classification_report(y_test_multi, y_pred_best))

# Матрица ошибок
cm = confusion_matrix(y_test_multi, y_pred_best)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Short', 'Medium', 'Long'], yticklabels=['Short', 'Medium', 'Long'])
plt.title('Матрица ошибок (лучший многоклассовый классификатор)')
plt.xlabel('Предсказанный класс')
plt.ylabel('Истинный класс')
plt.savefig('confusion_matrix_multi.png')
plt.close()

# Анализ матрицы
print("\nАнализ матрицы ошибок:")
print("Диагональные элементы показывают правильные предсказания.")
print("Вне диагонали — ошибки, например, Short, предсказанный как Medium.")


Лучшее k для многоклассовой классификации: 18

Метрики лучшего классификатора:
Точность (Accuracy): 0.5467
Точность (Precision): 0.5375
Полнота (Recall): 0.5467
F1-мера: 0.5380

Отчет классификации:
              precision    recall  f1-score   support

        Long       0.36      0.43      0.39        86
      Medium       0.41      0.31      0.35       110
       Short       0.82      0.89      0.86       104

    accuracy                           0.55       300
   macro avg       0.53      0.54      0.53       300
weighted avg       0.54      0.55      0.54       300


Анализ матрицы ошибок:
Диагональные элементы показывают правильные предсказания.
Вне диагонали — ошибки, например, Short, предсказанный как Medium.


## Выводы

- **Предобработка**: Пропуски в `margin_low` заполнены медианой.
- **Нормализация**: Улучшает производительность kNN за счет стандартизации масштабов признаков.
- **Отбор признаков**: Метод на основе корреляций (( T_{corr} = 0.3 ), ( T_{filter} = 0 )) позволяет выбрать наиболее информативные признаки.
- **Бинарная классификация**: Высокая точность, особенно после нормализации и фильтрации.
- **Многоклассовая классификация**: Искусственные классы на основе `length` дают умеренную точность; матрица ошибок показывает, какие классы путаются.
- **Кросс-валидация**: Позволяет выбрать оптимальное ( k ), балансируя переобучение и недообучение.